In [1]:
import os, sys, time
import numpy as np
import scipy.ndimage, scipy.misc
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
slim = tf.contrib.slim

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1"



def unpickle(file):
    with open(file, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
        return data

def load_cifar10_data(data_dir):
    train_data = None
    train_labels = []
    
    for i in range(1,6):
        data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
        if i==1:
            train_data = data_dic['data']
        else:
            train_data = np.vstack((train_data, data_dic['data']))
        train_labels += data_dic['labels']
    test_data_dic = unpickle(data_dir + "/test_batch")
    test_data = test_data_dic['data']
    test_labels = test_data_dic['labels']
    
    train_data = train_data.reshape((len(train_data), 3, 32, 32))
    train_data = np.rollaxis(train_data, 1, 4)
    train_labels = np.array(train_labels)
    
    test_data = test_data.reshape((len(test_data), 3, 32, 32))
    test_data = np.rollaxis(test_data, 1, 4)
    test_labels = np.array(test_labels)
    
    return train_data, train_labels, test_data, test_labels

data_dir = '/ideaHome/Dropbox/SJ/ML/Cifar10/Data/cifar-10-batches-py'
trImg, train_labels, teImg, test_labels = load_cifar10_data(data_dir)


print("Training data set : %3d, Test data Set : %3d" %(trImg.shape[0], teImg.shape[0]))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Training data set : 50000, Test data Set : 10000


In [2]:
def toLR(image, scale=2.):
    if len(image.shape)==4:
        num_sample = image.shape[0]
        images = np.zeros([image.shape[0], int(image.shape[1]/scale), int(image.shape[2]/scale), image.shape[3]])
        for i in range(num_sample):
            images[i,:,:,:] = scipy.misc.imresize(image[i,:,:,:], 1/scale,'bicubic')
        return images
    else:
        return scipy.misc.imresize(image, 1/scale, 'bicubic')
    

def Bicubic(image, scale=2):
    if len(image.shape)==4:
        bicImg = scipy.ndimage.interpolation.zoom(image, [1, scale, scale, 1], prefilter=False)
    else:
        bicImg = scipy.ndimage.interpolation.zoom(image, [scale,scale,1], prefilter=False)
    return bicImg

def _phase_shift(I, r):
    # Helper function with main phase shift operation
    bsize, a, b, c = I.get_shape().as_list()
    X = tf.reshape(I, (-1, a, b, r, r))
    X = tf.transpose(X, (0, 1, 2, 4, 3))  # bsize, a, b, 1, 1
    X = tf.split(X, a, 1)  # a, [bsize, b, r, r]
    X = tf.concat([tf.squeeze(x) for x in X], axis=2)  # bsize, b, a*r, r
    X = tf.split(X, b, 1)  # b, [bsize, a*r, r]
    X = tf.concat([tf.squeeze(x) for x in X], axis=2)  # bsize, a*r, b*r
    return tf.reshape(X, (-1, a*r, b*r, 1))

def PS(X, r, color=False):
  # Main OP that you can arbitrarily use in you tensorflow code
    if color:
        Xc = tf.split(X,3,3) #(3, 3, X)
        X = tf.concat([_phase_shift(x, r) for x in Xc], axis=3)
    else:
        X = _phase_shift(X, r)
    return X

def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis, keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
    return squash_factor * unit_vector

def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

In [3]:
class SRCapsNet(object):
    def __init__(self, mode='bicubic', color=True):
        self.LR_dim = (16,16,3)
        self.HR_dim = (32,32,3)
        self.batch_size = 1
        self.mode = mode
        self.color = color
        
        self.routing_iter = 2
        self.caps1_num = 16*16* 8
        self.caps1_dim = 8
        self.caps2_vec = 8
        self.caps2_num = 16*16* self.caps2_vec
        self.caps2_dim = 8
        
        self.W_init = tf.random_normal(shape=(1, self.caps1_num, self.caps2_num, self.caps2_dim, self.caps1_dim), 
                                      stddev = 0.1, dtype=tf.float32, name='W_init')
        self.W = tf.Variable(self.W_init, name='w')
        
        print('The model is generated')
        
    def model(self, img):
        with slim.arg_scope([slim.conv2d],kernel_size=[5,5], stride=[1,1], activation_fn = tf.nn.leaky_relu, padding='SAME',
                                                   weights_initializer=tf.contrib.layers.xavier_initializer()):
            batch_size = tf.shape(img)[0]
            self.conv1 = slim.conv2d(inputs=img, num_outputs=128, scope='conv1')
            self.conv2 = slim.conv2d(inputs=self.conv1, num_outputs=64, scope='conv2')
        self.caps1 = tf.reshape(self.conv2, [batch_size, self.caps1_num, self.caps1_dim], name='caps1')
        self.caps1_squash = squash(self.caps1, name='caps1_squash')


        self.W_tiled = tf.tile(self.W, [batch_size, 1, 1, 1, 1], name='W_tiled')

        self.caps1_expanded = tf.expand_dims(self.caps1_squash, -1, name='caps1_expanded')
        self.caps1_expanded2 = tf.expand_dims(self.caps1_expanded, 2, name='caps1_expanded2')
        self.caps1_tiled = tf.tile(self.caps1_expanded2, [1, 1, self.caps2_num, 1, 1], name='caps1_tiled')
        self.caps2_in = tf.matmul(self.W_tiled, self.caps1_tiled, name='caps2_in')
        
        self.b_ij = tf.zeros([batch_size, self.caps1_num, self.caps2_num, 1, 1], name='b_ij')
        for i in range(self.routing_iter):
            if i>0:
                self.b_ij = tf.add(self.b_ij, agreement)
            self.c_ij = tf.nn.softmax(self.b_ij, dim=2, name='c_ij')
            self.s_j = tf.reduce_sum(tf.multiply(self.c_ij, self.caps2_in), axis=1, keep_dims=True, name='s_j')
            self.v_j = squash(self.s_j, axis=-2, name='v_j')
            self.v_j_tiled = tf.tile(self.v_j, [1, self.caps1_num, 1, 1, 1], name='v_j_tiled')

            agreement = tf.matmul(self.caps2_in, self.v_j_tiled, transpose_a=True, name='agreement')

        self.caps2_out = self.v_j
        self.caps2_reshape = tf.reshape(self.caps2_out, [batch_size, 16, 16, self.caps2_vec*self.caps2_dim])
        with slim.arg_scope([slim.conv2d],kernel_size=[5,5], stride=[1,1], activation_fn = tf.nn.leaky_relu, padding='SAME',
                                                   weights_initializer=tf.contrib.layers.xavier_initializer()):
            self.conv3 = slim.conv2d(inputs=self.caps2_reshape, num_outputs=128, scope='conv3', activation_fn=tf.nn.leaky_relu)
            self.conv4 = slim.conv2d(inputs=self.conv3, num_outputs=12, scope='conv4', activation_fn=tf.nn.sigmoid)
        self.out_layer = PS(self.conv4, 2, self.color)
            
        out = self.out_layer
        return out
    
    def loss(self, SR, HR):
        loss = tf.reduce_mean(tf.square(SR - HR), name='loss')
        return loss
    
    def build(self):
        if self.color == True:
            if self.mode == 'bicubic':
                self.LRImg = tf.placeholder(tf.float32, [None, 32, 32, 3], 'LRImgBicubic')
            elif self.mode == 'pixelshuffle':
                self.LRImg = tf.placeholder(tf.float32, [None, 16, 16, 3], 'LRImgPixelShuffle')
            else:
                print ('undefined mode')
            self.HRImg = tf.placeholder(tf.float32, [None, 32, 32, 3], 'HRImg')
        else:
            if self.mode == 'bicubic':
                self.LRImg = tf.placeholder(tf.float32, [None, 32, 32, 1], 'LRImgBicubic')
            elif self.mode == 'pixelshuffle':
                self.LRImg = tf.placeholder(tf.float32, [None, 16, 16, 1], 'LRImgPixelShuffle')
            else:
                print ('undefined mode')
            self.HRImg = tf.placeholder(tf.float32, [None, 32, 32, 1], 'HRImg')
        self.SRImg = self.model(self.LRImg)
        self.LOSS = self.loss(self.SRImg, self.HRImg)
    
   
    
    

In [4]:
tf.reset_default_graph()
model = SRCapsNet('pixelshuffle', False)
model.build()

t_vars = tf.trainable_variables()
slim.model_analyzer.analyze_vars(t_vars, print_info=True)


The model is generated
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
---------
Variables: name (type shape) [size]
---------
w:0 (float32_ref 1x2048x2048x8x8) [268435456, bytes: 1073741824]
conv1/weights:0 (float32_ref 5x5x1x128) [3200, bytes: 12800]
conv1/biases:0 (float32_ref 128) [128, bytes: 512]
conv2/weights:0 (float32_ref 5x5x128x64) [204800, bytes: 819200]
conv2/biases:0 (float32_ref 64) [64, bytes: 256]
conv3/weights:0 (float32_ref 5x5x64x128) [204800, bytes: 819200]
conv3/biases:0 (float32_ref 128) [128, bytes: 512]
conv4/weights:0 (float32_ref 5x5x128x12) [38400, bytes: 153600]
conv4/biases:0 (float32_ref 12) [12, bytes: 48]
Total size of variables: 268886988
Total bytes of variables: 1075547952


(268886988, 1075547952)

In [5]:
learning_rate = 1e-5
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(model.LOSS)
print('learning rate is {}'.format(learning_rate))

learning rate is 1e-05


In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config) 
sess.run(tf.global_variables_initializer())
print('initialization complete')

initialization complete


In [7]:
batch_size=model.batch_size
print('batch_size: {}'.format(batch_size))
total_iter = int(trImg.shape[0]/batch_size)-1
for epoch in range(100):
    avg_psnr = 0
    start_time = time.time()
    for batch in range(total_iter):
        LRImg = toLR(trImg[batch:batch + batch_size,:,:,:])
        HRImg = trImg[batch:batch+batch_size,:,:,1:2]
        _, loss = sess.run([optm, model.LOSS], feed_dict={model.LRImg:LRImg[:,:,:,1:2], model.HRImg: HRImg})
        psnr = 20*np.log10(255./np.sqrt(loss))
        print ("\rbatch: {}/{} ({:.1f}%) psnr: {:.5f}".format(batch, total_iter, batch*100/total_iter, psnr), end="")
        avg_psnr+=psnr
    print ('\repoch: %3d, avg_PSNR: %4f, time for epoch: %.2f' %(epoch, avg_psnr/total_iter, time.time()-start_time))

batch_size: 1


/usr/local/lib/python3.5/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.5/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


epoch:   0, avg_PSNR: 5.882809, time for epoch: 106220.92
epoch:   1, avg_PSNR: 5.883219, time for epoch: 105397.92
epoch:   2, avg_PSNR: 5.883219, time for epoch: 102180.06
epoch:   3, avg_PSNR: 5.883219, time for epoch: 102551.13
batch: 6035/49999 (12.1%) psnr: 8.46017

KeyboardInterrupt: 

In [8]:
C=sess.run(model.out_layer, feed_dict={model.LRImg:LRImg})
plt.figure(figsize=[15,10])
plt.subplot(1,3,1)
plt.imshow(HRImg[0,:,:,:], cmap='gray')
plt.subplot(1,3,2)
plt.imshow(LRImg[0,:,:,1], cmap='gray')
plt.subplot(1,3,3)
plt.imshow(C[0,:,:,1], cmap='gray')


ValueError: Cannot feed value of shape (1, 16, 16, 3) for Tensor 'LRImgPixelShuffle:0', which has shape '(?, 16, 16, 1)'